# Multiple Linear Regression from Scratch*

*Daniel Wiesenfeld*

In this notebook I demostrate how to fit a multiple linear regression and duplicate every metric in `statsmodels`' linear regression summary output from scratch*. By that, I mean using only Python with the `numpy` librarby, and for certain metrics I also needed the `scipy` library. I have also created an accompanying Google Sheet and Excel Workbook that computes all the metrics (except one) using built-in formulas only - no plugins, no add-ons, no custom code. Though this exercise is mostly for the fun of it (and to learn how all the formulas work), there are actually practical uses. I think you will find that it is far simpler and easier to generate $\hat{\beta}$'s with just `numpy`, which you will almost always use anyway in a data science context, than it is using `sklearn` or `statsmodels`. Also, many businesses use spreadsheets all the time. I can think of many instances where having that functionality within a spreadsheet, without the need for plugins, add-ons, or custom programming can be very helpful.

Enjoy!

## Refresher on Multiple Linear Regression

If you are already an expert on multiple linear regression, feel fre to skip this part. If not, or if you're just a bit rusty, keep reading.

Multiple Linear Regression is a model that finds a linear function of a set of features $X$ that estimates $y$. The feature matrix $X$ has $n$ rows, one per observation, and $p$ columns, one per feature. The target vector $y$ has $n$ elements, one per observation. In the field of machine learning, the terms features and target are often used, while in the field of statistics, $X$ and $y$ are often called, independent variables and the dependent variable, or exogenous variables and endogenous variable. Regardless, the meaning stays the same. 

Multiple Linear Regression differs from Simple Linear Regression in that there can be more than one column or feature in $X$. In simple linear regression, $X$ is just a vector of $n$ elements, and the regression function can be thought of as a line geometrically. the two coefficients, $\beta_0$ and $\beta_1$, are the y-intercept and the slope of the line.

When $X$ has two or more features (excluding the constant, which we'll get to later), it becomes Multiple Linear Regression. When there are strictly two columns, the regression function can be thought of as a plane geometrically. The first coefficient, $\beta_0$, represents the y-intercept (the point on the y axis where the plane intersects it) and the next two coeficients, $\beta_1$ and $beta_2$ represent the slopes of the plane with respect to $y$ in the $x_1$ and $x_2$ axes respectively. Once there are three or more features, it becomes a hyperplane which is extremely difficult to visualize in three dimensions, but the concepts remain the same.

In order to fit the hyperplane, the algorithm finds the hyperplane that minimizes the sum of the square of the distances from each point to the hyperplane along the y axis. This is why the alogirthm is often referred to as "least squares". The implicit assumption is that our $y$ is a random variable that follows a very particular set of rules: It is a linear function of $X$ plus some normally distributed error that has a mean of 0 and a variance $\sigma^2$. The greater the magnitude of a coefficent, the more of an effect that feature has on $y$, and the sign of the coefficient indicates whether the effect is positive or negative. The lower $\sigma^2$ is, the more accurately the output of the linear function matches $y$.

This function is often written in various mathematically equivalent ways. The first way uses subscripts to represent the dimensional elements:

$$y_i = \beta_0 + \beta_1x_{i,1} + \beta_2x_{i,2} + \ldots + \beta_px_{i,p} + r_i \sim \mathcal{N}(0, \sigma^2)\quad\forall\; i \in \{1, \ldots, n\}$$

*Note: Many notations use the letter e to represent the error term, but I prefer r (residuals) so that it isn't confused with the base of the natural logarithm*

Another more concise approach uses summation over the features:

$$y_i = \beta_0 + \sum_{j = 1}^p \beta_jx_{i,j}+ r_i \sim \mathcal{N}(0, \sigma^2)\quad\forall\; i \in \{1, \ldots, n\}$$

The most concise and technical notation puts the whole thing into vector form, but before we do that let's deal with that pesky $\beta_0$ which ruins the pattern. All the other $\beta$'s are multiplied by their corresponding $x$'s, but the $\beta_0$ is all by itself. We can fix this with a simple mathematical trick. Anything times 1 is just itself. So, we can add a column to the beginning of $X$ (before the first column), that consists of just 1's. This new column is usually referred to as the "constant". Now we can rewrite the first version as:

$$y_i = \beta_0x_{i,0} + \beta_1x_{i,1} + \beta_2x_{i,2} + \ldots + \beta_px_{i,p} + r_i \sim \mathcal{N}(0, \sigma^2)\quad\forall\; i \in \{1, \ldots, n\}$$,

or the second version as:

$$y_i = \sum_{j = 0}^p \beta_jx_{i,j}+ r_i \sim \mathcal{N}(0, \sigma^2)\quad\forall\; i \in \{1, \ldots, n\}$$.

That didn't change anything mathematically, because the zero'th column of $X$ is just list of $n$ 1's. So each $\beta_0$ is just being multiplied by 1 and staying the same as before. It does however much more convenient to write in matrix form: 

$$y = X\beta^\top + r \sim \mathcal{N}(0, \sigma^2)$$

In the above notation, $X$ is an $n\times p$ matrix (this new $p$ is one more than the previous $p$ because we added the constant), $\beta$ is a $p$ length vector, and $y$ and $r$ are both $n$ length vectors.


Now this function of $y$ is unknown. We don't know the true values of $\beta$ and we don't no the true value of $\sigma^2$. The least squares algorithm allows us to estimate those values. We commonly notate estimated values by putting little hats on them. So our estimated values would be $\hat\beta$ and $\hat\sigma^2$. It is also common to refer to $\hat\beta$ as $b$.

Once we have our $b$ values, we can use them on any row of $x$ values to estimate a value for its corresponding $y$ value - which also gets a hat: $\hat y$. When we do this with a previously unseen row or rows of $x$ values, those computed valeus of $\hat y$ can be thought of as predicted values of $y$.



## Imports and Data Generation

Let's import our libraries and get the data in a `pandas` dataframe.

In [534]:
from statsmodels.api import OLS, datasets, add_constant # we'll use this to get some data and to compare our results

import numpy as np
import scipy as sp
import pandas as pd

Let's use the macrodata dataset from `statsmodels` as our example. We will build a multiple regression model to predict unemployment with seven features.

In [535]:
df = pd.DataFrame(datasets.macrodata.load().data)
df.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


We'll create our `X` dataframe, by selecting 7 features of interest and adding a constant, and we will select the 'unemp' column as our `y` series.

In [536]:
X = df[['realgdp', 'realcons', 'realgovt', 'realdpi', 'cpi', 'm1', 'pop']].pipe(add_constant)
X.head()

,const,realgdp,realcons,realgovt,realdpi,cpi,m1,pop
0,1.0,2710.349,1707.4,470.045,1886.9,28.98,139.7,177.146
1,1.0,2778.801,1733.7,481.301,1919.7,29.15,141.7,177.830
2,1.0,2775.488,1751.8,491.260,1916.4,29.35,140.5,178.657
3,1.0,2785.204,1753.7,484.052,1931.3,29.37,140.0,179.386
4,1.0,2847.699,1770.5,462.199,1955.5,29.54,139.6,180.007


In case you want to be a stickler and say I can't use statsmodels to add the constant, just use this cell instead:

In [537]:
X = df[['realgdp', 'realcons', 'realgovt', 'realdpi', 'cpi', 'm1', 'pop']]
X.insert(0, 'const', 1.0)

X.head()

,const,realgdp,realcons,realgovt,realdpi,cpi,m1,pop
0,1.0,2710.349,1707.4,470.045,1886.9,28.98,139.7,177.146
1,1.0,2778.801,1733.7,481.301,1919.7,29.15,141.7,177.830
2,1.0,2775.488,1751.8,491.260,1916.4,29.35,140.5,178.657
3,1.0,2785.204,1753.7,484.052,1931.3,29.37,140.0,179.386
4,1.0,2847.699,1770.5,462.199,1955.5,29.54,139.6,180.007


and let's create our y vector

In [538]:
y = df['unemp']
y.head()

0    5.8
1    5.1
2    5.3
3    5.6
4    5.2
Name: unemp, dtype: float64

If you want to try out the Excel sheets on your own, run the following code if you want to copy X & y as a single dataframe to the clopboard so you can paste in Sheets or Excel:

In [539]:
X.assign(y = y).to_clipboard()
# now just paste in Sheets or Excel

## Statsmodels Output

OK let's take a look at what `statsmodels` gives us

In [540]:
mod = OLS(y, X)
res = mod.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  unemp   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     197.6
Date:                Thu, 18 May 2023   Prob (F-statistic):           6.21e-85
Time:                        17:27:02   Log-Likelihood:                -151.94
No. Observations:                 203   AIC:                             319.9
Df Residuals:                     195   BIC:                             346.4
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -17.5391      2.806     -6.250      0.000     -23.074     -12.004
realgdp       -0.0116      0.000    -27.105      0.000      -0.012      -0.011
realcons       0.0087      0.001     10.055      0.000       0.007       0.010
realgovt      -0.0030      0.001     -4.824      0.000      -0.004      -0.002
realdpi        0.0029      0.001      4.112      0.000       0.002       0.004
cpi            0.0799      0.006     13.065      0.000       0.068       0.092
m1            -0.0030      0.001     -4.945      0.000      -0.004      -0.002
pop            0.1907      0.018     10.646      0.000       0.155       0.226
==============================================================================
Omnibus:                        0.415   Durbin-Watson:                   0.779
Prob(Omnibus):                  0.813   Jarque-Bera (JB):                0.204
Skew:                           0.057   Prob(JB):                        0.903
Kurtosis:                       3.106   Cond. No.                     8.61e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.61e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

The problem with `statsmodels` output is that the numbers are all rounded to no more than three decimal places. It will be hard to see how accurate our own computations are unless we extract these attributes independently from the `res` object. I show you how to do it below and conveniently save them in variables beginning with the suffix '_sm', so that we can compare them to our own computations.

In [541]:
r2_sm = res.rsquared # R-squared
r2_sm

0.8764203251692085

In [542]:
r2_adj_sm = res.rsquared_adj # Adj. R-squared
r2_adj_sm

0.8719841317137442

In [543]:
f_sm = res.fvalue # F-Statistic
f_sm

197.56134036257416

In [544]:
p_f_sm = res.f_pvalue # Prob(F-Statistic)
p_f_sm

6.214397666559647e-85

In [545]:
l_sm = res.llf # Log-Likelihood
l_sm

-151.94432883403306

In [546]:
aic_sm = res.aic # AIC
aic_sm

319.8886576680661

In [547]:
bic_sm = res.bic # BIC
bic_sm

346.39430550040043

In [548]:
b_sm = res.params # coefficients
b_sm

const      -17.539104
realgdp     -0.011650
realcons     0.008694
realgovt    -0.003007
realdpi      0.002923
cpi          0.079861
m1          -0.003043
pop          0.190674
dtype: float64

In [549]:
s_sm = res.bse # standard erros of coefficients
s_sm

const       2.806322
realgdp     0.000430
realcons    0.000865
realgovt    0.000623
realdpi     0.000711
cpi         0.006113
m1          0.000615
pop         0.017910
dtype: float64

In [550]:
t_sm = res.tvalues # t values of coefficients
t_sm

const       -6.249854
realgdp    -27.105072
realcons    10.054683
realgovt    -4.824153
realdpi      4.112236
cpi         13.064576
m1          -4.944581
pop         10.646391
dtype: float64

In [551]:
p_sm = res.pvalues # p values of coefficients
p_sm

const       2.532927e-09
realgdp     4.704738e-68
realcons    1.991447e-19
realgovt    2.828657e-06
realdpi     5.772642e-05
cpi         1.972339e-28
m1          1.641259e-06
pop         3.691092e-21
dtype: float64

In [552]:
lb_sm = res.conf_int()[0] # lower bound of 95% confidence interval of coefficients
lb_sm

const      -23.073744
realgdp     -0.012497
realcons     0.006989
realgovt    -0.004236
realdpi      0.001521
cpi          0.067805
m1          -0.004256
pop          0.155352
Name: 0, dtype: float64

In [553]:
ub_sm = res.conf_int()[1] # upper bound of 95% confidence interval of coefficients
ub_sm

const      -12.004463
realgdp     -0.010802
realcons     0.010399
realgovt    -0.001777
realdpi      0.004325
cpi          0.091916
m1          -0.001829
pop          0.225995
Name: 1, dtype: float64

The bottom-most table of the summary output requires a few more methods from `statsmodels` and access to the model's residuals.

In [554]:
from statsmodels.stats.stattools import omni_normtest, robust_skewness, robust_kurtosis, durbin_watson, jarque_bera
resid_sm = res.resid # residuals (errors)

In [555]:
omni_sm = omni_normtest(resids_sm)[0] # Omnibus statistic of residuals
omni_sm

0.41478658948228114

In [556]:
p_omni_sm = omni_normtest(resids_sm)[1] # Prob(Omnibus)
p_omni_sm

0.8126999565057904

In [557]:
skew_sm = robust_skewness(resids)[0] # Skewness of residuals
skew_sm

0.056667302727931364

In [558]:
kurt_sm = robust_kurtosis(resids)[0] + 3 # Kurtosis of residuals
kurt_sm

3.1061908834830367

In [559]:
dw_sm = durbin_watson(resids) # Durbin-Watson
dw_sm

0.778964211500831

In [560]:
jb_sm = jarque_bera(resids)[0] # Jarque-Bera (JB)
jb_sm

0.20402545897229157

In [561]:
p_jb_sm = jarque_bera(resids)[1] # Prob(JB)
p_jb_sm

0.9030180566398727

In [562]:
cn_sm = res.condition_number #Cond. No.
cn_sm

860663.3059185082

Phew!

## Coding the Metrics from Scratch

### Coefficients

Let's start with the most important metrics, the coefficients themselves!
Multiple Linear Regression is actually just a linear algebra problem. Pro tip: the `@` operator is a shortcut form matrix multiplication. In multiple linear regression, we assume that  $ y = \beta^\top X + e$, where $e$ is an n - length vector of in which each $e_i$ is a normally distributed random variable with mean $0$ and variance $\sigma^2$. We don't know the true value of $\beta$, but we can estimate $\hat{\beta}$ or $b$ for short as:

$$b = (X^\top X)^{-1}X^\top y $$

In [563]:
b = np.linalg.inv(X.T @ X) @ X.T @ y
b.index = X.columns #reindexing with column names makes it easier to see!
b

const      -17.539104
realgdp     -0.011650
realcons     0.008694
realgovt    -0.003007
realdpi      0.002923
cpi          0.079861
m1          -0.003043
pop          0.190674
dtype: float64

That's it! let's compare them to the coefficients we pulled from `statsmodels`.

In [564]:
b - b_sm

const      -2.014140e-10
realgdp    -1.008395e-14
realcons    9.141299e-14
realgovt    2.271707e-14
realdpi    -9.579967e-14
cpi         1.537936e-13
m1         -1.881134e-14
pop         1.316003e-12
dtype: float64

Ok so we're off by a teeny tiny fraction ... most likely due to rounding differences between `statsmodels` and `numpy`.

The standard errors are a bit more involved, so let's jump up to the top table of the summary and compute some things there first, as some of them will come in handy later.

### The Top Table

The top table contains information pertaining to the entire model. We won't bother computing things like dependent variable name, model type, date, and time because those are trivial and obvious, but we'll compute everything else. 

We'll start with the three values in the first colum, the number of observations and the two degrees of freedom.

*Note: the covariance type is always non-robust by default and our computations assume a non-robust covariance, so we will not get into that either.*

In [565]:
# The number of observations is simply the number of rows in X or the length of y, which we'll call n.
n = len(y)
n

203

In [566]:
# The degress of freedom of the residuals is n less the number of parameters of the model,
# and the degrees of freedom of the model is the number of parameters of the model - 1.

# We'll start by creating p as the parameters of the model, which is just the columns in X, and then compute the two df's.
# These degrees of freedoms are used in various other metrics. The variable p will come in handy later.

p = X.shape[1]
dfr = n - p
dfm = p - 1
dfr, dfm

(195, 7)

Now let's compute R-squared. R-squared tells us how well $X$ explains $y$. It ranges from 0, no explanatory power, to 1, full explanatory power. It is computed as: 
$$R^2 = 1 - \frac{\text{SSE}}{\text{SST}}$$


SSE or sum of squares of errors is the sum of the squares of each residual (error) $r_i$ - the difference between each true value $y_i$ and its predicted value $\hat{y_i}$:
$$ \text{SSE} = \sum_{i=1}^n r_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$


SST or the sum of total squares is the sum of the squares of of the differences between each $y_i$ and $\bar{y}$ (the average of all $y_i$'s):

$$ \text{SST} = \sum_{i=1}^n (y_i - \bar{y})^2.$$

In [567]:
# let's compute all the things we'll need and the R-squared itself.

yhat = X @ np.array(beta).T # for whatever reason, we need to first recast the beta series as an array before the matrix mult
resid = y - yhat
sse = (resid**2).sum()
ybar = y.mean()
sst = ((y - ybar)**2).sum()
r2 = 1 - sse/sst
r2

0.8764203251692086

In [568]:
# and let's compare to sm
r2 - r2_sm

1.1102230246251565e-16

Adjusted R-squared is just a version of R squared that is penalized for having additional parameters. As you can see for a two-parameter model (i.e. simple linear regression), it reduces to R-squared:

$$ R^2_{\text{adj.}} = 1 - \frac{\text{SSE}}{\text{SST}}\left(\frac{n - 1}{n-p-1}\right) $$



In [569]:
r2_adj = 1 - (sse/sst) * (n - 1)/(n - p - 1)
r2_adj

0.8713242561040214

The F test measures the probability that at least one independent variable (excluding the constant) makes the model more predictive than not having any of the independent variables. Essentially, it's the probability that our model beats the baseline estimator $\bar{y}$. The greater the F-statistic, the better our model. Our null hypothesis is that our model is no better than $\bar{y}$. In order to put a probability to that, we take the area under the right tail of the F distribution (i.e. where x > F-stat) with the degrees of freedom we computed above. That value is the probability of incorrectly rejecting the null hypothesis (i.e. believing that our model beats the baseline when it actually doesn't). You can also thinkg of is as the probability that we would see results as extreme or more extreme than our model results by chance even if there was in reality no relationship between the independent variables and $y$. The smaller the p-value, the more confident we can be in our model. 

The F-Statistic is computed as:

$$\text{F-statistic} = \frac{\text{MSM}}{\text{MSE}} \sim \mathcal{F}(\text{df}_{\text{residual}},\text{df}_{\text{model}} )$$

The denominator of that, the MSE, is the mean version of hte SSE we computed above and is computed as: 
$$ \text{MSE} = \frac{\text{SSE}}{\text{df}_{\text{residual}}}$$,
and if you recall $\text{df}_{\text{residual}} = n-p$.

The numerator, the MSM, is the mean version of the SSM, sum of squares model, which we have not yet computed. The SSM is the sum of the squares of the differences between each $\hat{y_i}$ and $\bar{y}$. You can think of it as how much we gain from the model vs. our baseline guess. If we didn't have our model, and we just had all the values of $y$, if we had to guess the value of a randomly drawn $y_i$, our best guess would be the mean, $\bar{y}$:
$$\text{SSM} =\sum_{i=1}^n (\hat{y_i} - \bar{y})^2 $$

(Note that $\text{SST} = \text{SSE} + \text{SSM}$)

MSM is then computed as:
$$ \text{MSM} = \frac{\text{SSM}}{\text{df}_{\text{model}}} $$,
and if you recall, $\text{df}_{\text{model}} = p-1$.

In [570]:
ssm = ((yhat - ybar)**2).sum()
msm = ssm / dfm

mse = sse / dfr
f = msm / mse
f

197.5613403619848

In [571]:
# and let's compare to sm
f - f_sm

-5.893525667488575e-10

Now we compute the p-value of the F statistic using the F distribution

In [572]:
p_f = sp.stats.f.sf(f, dfm, dfr)
p_f

6.214397668138137e-85

In [573]:
# and let's compare to sm
p_f - p_f_sm

1.57849073308343e-94

The Log-Likelihood is a measure of how likely the coefficients estimates in our model are the *true* coefficients. Multiple regression assumption assumes that: $ y = \beta^\top X + r \sim \mathcal{N}(0, \sigma^2)$, where $r$ is an n - length vector in which each $r_i$ is a normally distributed random variable with mean $0$ and variance $\sigma^2$. We don't know the true coeffiecients $\beta$ or $\sigma^2$ of this  function, but the multiple regression algorithm allows us to estimate $\beta$ as $\hat{\beta}=b$ and estimate $\sigma$ as $\hat{\sigma}=mse$.

The Likelihood $\mathcal{L}$, represents the likelihood that our estimates are correct. It is the product of the value of the pdf of the Normal distribution evaluated at each observation of our training set using our estimated values. However, for mathematical convenience, we usually use the Log-Likelihood $\mathcal{l}$, which is equivalent to the sum of the log of the pdf of the Normal Distribution evaluated at each observation of our training set using our estimated parameters (because the log of a product is equivalent to the sum of logs - $\log(\Pi x_i) = \Sigma \log(x_i)$). While the full formula is complex, in the case of multiple regression, it reduces to relatively simple formula:

$$ \text{Log-Likelihood} = \mathcal{l} = \frac{n}{2}\left(\log(n) - \log(2\pi\: \text{SSE}) - 1\right) $$

The Log-Likelihood can take any real value from $-\infty$ to $+\infty$ - though in practice it will usually be negative. The greater the Log-Likelihood, the more confident we can be in our parameter estimates.

In [574]:
l = n/2 * (np.log(n) - np.log(2 * np.pi * sse) - 1)
l

-151.944328834033

In [575]:
# and let's compare to sm
l - l_sm

5.684341886080802e-14

AIC (Akaike Information Criterion) and BIC (Bayesian Information Criterion) both measure information gain by the model while penalize extra parameters (higher number is better).

$$\text{AIC} = 2p - 2\mathcal{l} $$
$$\text{BIC} = p\log(n) - 2\mathcal{l}$$


In [576]:
aic = 2 * p - 2 * ll
aic

319.888657668066

In [577]:
# and let's compare to sm
aic - aic_sm

-1.1368683772161603e-13

In [578]:
bic = p * np.log(n) - 2 * ll
bic

346.3943055004003

In [579]:
# and let's compare to sm
bic - bic_sm

-1.1368683772161603e-13

OK, let's return to the middle table of the summary, the coefficeint table.

### The Standard Errors

Each exogenous variable has a true, unknown $\beta_j$, so we estimated $b_j$ as our best guess. We did this many steps ago with the formula:

$$b = (X^\top X)^{-1}X^\top y$$,

which represents the estimated expected value of $\beta$, now we need to compute the standard error of each $b_j$ which is a measure of the expected deviation from the expectation (like a standard deviation), which gives us a sense of how confident we are in the coeficient estimate.

To do that we take the square root of the product of the MSE we computed above and each $b_j$'s corresponding diagonal element of $(X^\top X)^{-1}$ (the element at the $j$th row and $j$th column).

$$\text{standard error}_j = s_j = \sqrt{\text{MSE}\times(X^\top X)^{-1}_{jj}}$$


In [580]:
s = np.sqrt(mse * np.diag(np.linalg.inv(X.T @ X)))
s = pd.Series(name = 's', data = s, index = X.columns) #reindexing with column names makes it easier to see!
s

const       2.806322
realgdp     0.000430
realcons    0.000865
realgovt    0.000623
realdpi     0.000711
cpi         0.006113
m1          0.000615
pop         0.017910
Name: s, dtype: float64

In [581]:
# and let's compare to sm
s - s_sm

const      -3.210765e-13
realgdp     9.164761e-16
realcons    3.199047e-15
realgovt   -1.128654e-16
realdpi     1.051568e-15
cpi         1.607221e-15
m1          7.697835e-17
pop         7.719519e-15
dtype: float64

The T-value, is the ratio between the coefficient and its standard error. 
$$ T = \frac{b}{se} \sim \mathcal{t}_{\text{df}_{\text{residual}}} $$

We then use each one to determine the probability that the we will incorrectly reject the null hypothesis. That is, the probability that the coefficient is, in reality 0. This is eqivalent to a draw from Student's $t$ distribution with df$_{residual}$ degrees of freedom that has a magnitude greater than that of the T value's magnitude. In other words, the probability that a draw from the distribution is greather than the absolute value of the T value or less than the negation of the absolute value of the T value. This is known as a two-tailed test. The smaller the p value for a coefficient, the more confident we can be that the true value of the coefficient is not 0.
$$ p = \mathbb{P}\left[(x \sim t_{\text{df}_{residual}} < - |T|) \cup (x \sim t_{\text{df}_{residual}} > |T|)\right] $$

We can also determine the confidence interval around the coefficient for any significance $1 - \alpha$ (95% by default) by using the critical value of the t distribution that leaves $\alpha/2$ probability mass in each tail.

In [582]:
t = b/se
t

const       -6.249854
realgdp    -27.105072
realcons    10.054683
realgovt    -4.824153
realdpi      4.112236
cpi         13.064576
m1          -4.944581
pop         10.646391
dtype: float64

In [583]:
# and let's compare to sm
t - t_sm

const      -7.248691e-11
realgdp     3.433342e-11
realcons    6.852119e-11
realgovt    3.557599e-11
realdpi    -1.408402e-10
cpi         2.172307e-11
m1         -2.995115e-11
pop         6.889067e-11
dtype: float64

In [584]:
p = t.apply(lambda x: sp.stats.t.sf(abs(x), dfr)*2)
p

const       2.532927e-09
realgdp     4.704738e-68
realcons    1.991447e-19
realgovt    2.828657e-06
realdpi     5.772642e-05
cpi         1.972339e-28
m1          1.641259e-06
pop         3.691092e-21
dtype: float64

In [585]:
# and let's compare to sm
p - p_sm

const      -9.791399e-19
realgdp     9.194666e-78
realcons   -9.122227e-29
realgovt    4.508604e-16
realdpi     3.240156e-14
cpi        -3.002023e-38
m1         -2.241380e-16
pop        -1.726683e-30
dtype: float64

In [586]:
significance = .95
alpha = 1 - significance
t_critical = sp.stats.t.isf(alpha/2, dfr)
lb = b - t_critical * se
lb

const      -23.073744
realgdp     -0.012497
realcons     0.006989
realgovt    -0.004236
realdpi      0.001521
cpi          0.067805
m1          -0.004256
pop          0.155352
dtype: float64

In [587]:
# and let's compare to sm
lb - lb_sm

const      -2.007816e-10
realgdp    -1.189153e-14
realcons    8.510380e-14
realgovt    2.293998e-14
realdpi    -9.787375e-14
cpi         1.506295e-13
m1         -1.896400e-14
pop         1.300765e-12
dtype: float64

In [588]:
ub = b + t_critical * se
ub

const      -12.004463
realgdp     -0.010802
realcons     0.010399
realgovt    -0.001777
realdpi      0.004325
cpi          0.091916
m1          -0.001829
pop          0.225995
dtype: float64

In [589]:
# and let's compare to sm
ub - ub_sm

const      -2.020464e-10
realgdp    -8.276366e-15
realcons    9.772218e-14
realgovt    2.249438e-14
realdpi    -9.372537e-14
cpi         1.569578e-13
m1         -1.865934e-14
pop         1.331241e-12
dtype: float64

Ok, that wraps up the middle table of the summary, let's move on to the final table.

### The Final Table (Residual Analysis)

The final table is mostly focused on statistics related to the residuals (errors). These are diagnostics to measure the degree to which the errors are normally distributed and heteroskedastic. There is also a statistic focused on multicollinearity of the coefficients.

Let's start with the Skewness and Kurtosis. The Skewness is the third moment of a distrubution and measures its asymetry around the mean. A normal distrubution is perfectly symetric and has a skewness of 0. A positively skewed distribution has a longer right tail and its mean is greater than its median. A negatively skewed distribution has a longer left tail and its mean is less than its median.

$$\text{Skewness}_{residual} = \frac{\sum_{i = 1}^n r_i^3}{n \sigma_r^3}$$,

where $r$ is the vector of residuals and $\sigma_r$ is its standard deviation.

Kurtosis is the fourth moment of a distribution and measures its "spikiness" or "heaviness". A Normal distribution has a kurtosis of 3, so some versions subtract 3 from the kurtosis so that the normal baseline would be 0 and call that value "excess kurtosis". Sometimes excess kurtosis is just referred to as kurtoses, so it is important to know which version you are using. A normal distribution is knowns as mesokurtic. When kurtosis increases, the distribution becomes leptokurtic, meaning probability mass shifts into the tails, so the center of the distribution becomes thinner and taller. When the he kurtosis decreases the distribution becomes platykurtic, meaning the probability mass shifts toward the mean so the center becomes wider and shorter.


$$\text{Kurtosis}_{residual} = \frac{\sum_{i = 1}^n r_i^4}{n \sigma_r^4}$$,

where $r$ is the vector of residuals and $\sigma_r$ is its standard deviation.

In [590]:
skew = sum(resid**3)/(n * np.std(resid)**3)
skew

0.056667302773462505

In [591]:
# and let's compare to sm
skew - skew_sm

4.5531141357191274e-11

In [592]:
kurt = sum(resid**4)/(n * np.std(resid)**4)
kurt

3.106190883487652

In [593]:
# and let's compare to sm
kurt - kurt_sm

4.615419157971701e-12

One of the assumptions of linear regression is that the residuals are normally distributed. We the following two tests use the skewness and kurtosis of the residuals to measaure the probability that the residuals are actuall normally distributed. In each case the null hypothesis is that the skewness and excess kurtosis are both 0. The p value for each represents the probability of incorrectly rejecting the null hypothesis - that is assuming that the residuals are not normally distributed when in fact they are. Therefore, unlike the previous p-values, we are looking for high p-values here.

The first test is the Omnibus (also known as the D'Agostiono- Pearson) statistic. 

$$\text{Omnibus} = Z_s(\text{skewness})^2 + Z_k(\text{kurtosis})^2 \sim \chi^2_2$$

The $Z$ transoformations make the skewness and kurtoses normally distributed and are a bit involved to compute. I credit https://real-statistics.com/tests-normality-and-symmetry/statistical-tests-normality-symmetry/dagostino-pearson-test/ for breaking it down into several steps.

Once we have the Omnibus statistic, we can use a right-tailed $\chi^2$ test to see the probability that the distribution is not Normal.

In [594]:
#Note I add an underscore to variables where the name might conflict with an already used variable

c = 3 * (n**2 + 27 * n - 70) * (n + 1) * (n + 3) / ((n - 2) * (n + 5) * (n + 7) * (n + 9))
w2 = np.sqrt(2 * (c - 1)) - 1
w = np.sqrt(w2)
a = np.sqrt((w2 - 1)/2)
b_ = 1/np.sqrt(np.log(w))
u = a * skew * np.sqrt((n + 1) * (n + 3) / (6 * (n - 2)))
zs = b_ * np.log(u + np.sqrt(u**2 + 1))

d = np.sqrt((n + 1)**2 * (n + 3) * (n + 5)/(24 * n * (n - 2) * (n - 3)))
e = (6 * (n**2 - 5 * n + 2)/((n + 7) * (n + 9))) * np.sqrt(6 * (n + 3) * (n + 5)/(n * (n - 2) * (n - 3)))
f_ = 6 + (8 / e) * ((2 / e) + np.sqrt(1 + 4 / e**2))
g = d * (kurt - 3 * (n - 1)/ (n + 1)) * np.sqrt(2 / (f_ - 4))
v = (1 - 2 / f_)/(1 + g)
r_ = 2 / (9 * f_)
zk = (1 - r_ - v **(1 / 3))/np.sqrt(r)

omni = zs**2 + zk **2
omni

0.4147865896824052

In [595]:
# and let's compare to sm
omni - omni_sm

2.0012408397107606e-10

In [596]:
p_omni = sp.stats.chi2.sf(omni, 2)
p_omni

0.81269995642447

In [597]:
# and let's compare to sm
p_omni - p_omni_sm

-8.132039486241638e-11

The other test of normality shown in the summary is the Jarque-Bera test. The JB test is defined as follows:

$$\text{JB} = \frac{n}{6} \text{Skewness}^2  + \frac{(\text{Kurtosis} - 3)^2}{4} \sim \chi^2_2$$

In [598]:
jb = n/6 * (skew**2 + (kurt - 3)**2/4)
jb

0.20402545915517234

In [599]:
# and let's compare to sm
jb - jb_sm

1.828807660864129e-10

In [600]:
p_jb = sp.stats.chi2.sf(jb, 2)
p_jb

0.9030180565573004

In [601]:
# and let's compare to sm
p_jb - p_jb_sm

-8.25722823449837e-11

The top metric of the bottom table is the Durbin-Watson statistic. It measures autocorrelation in the residuals, which would violate the assumption of homoskedasticity (constant variance). A value of 2 means that there is no autocorrelation, 0-2 means negative autocorrelation, and 2-4 means positive autocorrelation. This is test is considered a bit outdated and is rarely used. 

$$\text{DW} = \frac{\sum_{i = 2}^n (r_i - r_{i - 1})^2}{\text{SSE}}$$

In [602]:
dw = (np.diff(resid)**2).sum()/sse
dw

0.7789642114989123

In [603]:
# and let's compare to sm
dw - dw_sm

-1.9186874311571955e-12

The last metric on the summary table is the Condition Number. This is actually not related to the residuals, so it kind of doesn't belong. It measures the degree to which the parameters of the model are intercollinear. This requires computing the eigenvalues of the $(X^\top X)$ matrix we used to compute the coefficient estimates and their standard errors. 

Let $\lambda_{max}$ and $\lambda_{min}$ be the maximum and minimum eigenvalues of $X^\top X$, respectively:

$$\text{Condition Number} = \sqrt{\frac{\lambda_{max}}{\lambda_{min}}}$$

*Note: There are some issues with the implementation of condition number in statsmodels, as it does not normalize or center the X values and includes the constant which can cause an artificially high condition number. I include it for completelness, but I won't get into it. I also do not include it in the Excel and Sheets version of this because I do not know a convenient way to compute eigenvalues in spreadsheets using just built-in formulas.*

In [604]:
eig = np.linalg.eigvalsh(X.T @ X)
cn = np.sqrt(max(eig)/min(eig))
cn

860663.3059171436

In [605]:
import datetime as dt     
now = dt.datetime.now()
today = now.strftime('%a, %d %b, %Y')
time = now.strftime('%H:%M:%S')

t1l = [('Dep. Variable:', y.name), 
        ('Model:', 'OLS'),
        ('Method:', 'Least Squares'),
        ('Date:', today),
        ('Time:', time),
        ('No. Observations:', n),
        ('DF Residuals:', dfr),
        ('DF Model:', dfm),
        ('Covariance Type:', 'nonrobust')]

t1r = [('R-squared:', r2), 
        ('Adj. R-squared:', r2_adj),
        ('F-statistic:', f),
        ('Prob (F-statistic):', p_f),
        ('Log-Likelihood:', l),
        ('AIC:', aic),
        ('BIC:', bic),
        ('', ''),
        ('', '')]

(t1c0, t1c1), (t1c2, t1c3) = (zip(*t1l), zip(*t1r))
t1 = pd.DataFrame(zip(t1c0, t1c1, t1c2, t1c3)).style\
    .set_caption("OLS Regression Results")\
    .hide(axis = 'index').hide(axis = 'columns')\
    .applymap(lambda x: "font-weight: bold", subset = [0, 2])\
    .format(precision=3)\
    .format(precision=1, subset = ([2, 5, 6], 3))\
    .format("{:.2e}", subset = (3, 3))

t1

Dep. Variable:,unemp,R-squared:,0.876
Model:,OLS,Adj. R-squared:,0.871
Method:,Least Squares,F-statistic:,197.6
Date:,"Thu, 18 May, 2023",Prob (F-statistic):,6.21e-85
Time:,17:28:04,Log-Likelihood:,-151.944
No. Observations:,203,AIC:,319.9
DF Residuals:,195,BIC:,346.4
DF Model:,7,,
Covariance Type:,nonrobust,,


In [662]:
alpha

0.050000000000000044

In [669]:
"[{:0.3}".format(alpha/2)

'[0.025'

In [670]:
t2 = pd.concat((pd.DataFrame([['coef', 'std err', 't', 'P>|t|', "{[:0.3}".format(alpha/2), "{[:0.3}".format(1 - alpha/2)]]), 
                pd.concat((b, se, t, p, lb, ub), axis = 1, ignore_index = True)))\
    .set_axis([''] + list(b.index), axis = 0).style\
    .hide(axis = 'columns')\
    .format(precision=3)\
    .format(precision=4, subset = [0])\
    .applymap(lambda x: "font-weight: bold", subset = ('', slice(6)))
t2

ValueError: expected '}' before end of string

In [660]:
t3l = [('Omnibus:', omni), 
       ('Prob(Omnibus):', p_omni),
       ('Skew:', skew),
       ('Kurtosis:', kurt)]

t3r = [('Durbin-Watson:', dw), 
       ('Jarque-Bera (JB):', jb),
       ('Prob(JB):', p_jb),
       ('Cond. No.:', cn)]

(t3c0, t3c1), (t3c2, t3c3) = (zip(*t3l), zip(*t3r))
t3 = pd.DataFrame(zip(t3c0, t3c1, t3c2, t3c3)).style\
    .hide(axis = 'index').hide(axis = 'columns')\
    .applymap(lambda x: "font-weight: bold", subset = [0, 2])\
    .format(precision=3)\
    .format("{:.2e}", subset = (3, 3))

t3

Omnibus:,0.415,Durbin-Watson:,0.779
Prob(Omnibus):,0.813,Jarque-Bera (JB):,0.204
Skew:,0.057,Prob(JB):,0.903
Kurtosis:,3.106,Cond. No.:,8.61e+05


In [661]:
display(t1, t2, t3)

Dep. Variable:,unemp,R-squared:,0.876
Model:,OLS,Adj. R-squared:,0.871
Method:,Least Squares,F-statistic:,197.6
Date:,"Thu, 18 May, 2023",Prob (F-statistic):,6.21e-85
Time:,17:28:04,Log-Likelihood:,-151.944
No. Observations:,203,AIC:,319.9
DF Residuals:,195,BIC:,346.4
DF Model:,7,,
Covariance Type:,nonrobust,,


,coef,std err,t,P>|t|,[0.025,0.975]
const,-17.5391,2.806,-6.250,0.000,-23.074,-12.004
realgdp,-0.0116,0.000,-27.105,0.000,-0.012,-0.011
realcons,0.0087,0.001,10.055,0.000,0.007,0.010
realgovt,-0.0030,0.001,-4.824,0.000,-0.004,-0.002
realdpi,0.0029,0.001,4.112,0.000,0.002,0.004
cpi,0.0799,0.006,13.065,0.000,0.068,0.092
m1,-0.0030,0.001,-4.945,0.000,-0.004,-0.002
pop,0.1907,0.018,10.646,0.000,0.155,0.226


Omnibus:,0.415,Durbin-Watson:,0.779
Prob(Omnibus):,0.813,Jarque-Bera (JB):,0.204
Skew:,0.057,Prob(JB):,0.903
Kurtosis:,3.106,Cond. No.:,8.61e+05


In [407]:
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  unemp   R-squared:                       0.876
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     197.6
Date:                Thu, 18 May 2023   Prob (F-statistic):           6.21e-85
Time:                        15:27:17   Log-Likelihood:                -151.94
No. Observations:                 203   AIC:                             319.9
Df Residuals:                     195   BIC:                             346.4
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -17.5391      2.806     -6.250      0.000     -23.074     -12.004
realgdp       -0.0116      0.000    -27.105      0.000      -0.012      -0.011
realcons       0.0087      0.001     10.055      0.000       0.007       0.010
realgovt      -0.0030      0.001     -4.824      0.000      -0.004      -0.002
realdpi        0.0029      0.001      4.112      0.000       0.002       0.004
cpi            0.0799      0.006     13.065      0.000       0.068       0.092
m1            -0.0030      0.001     -4.945      0.000      -0.004      -0.002
pop            0.1907      0.018     10.646      0.000       0.155       0.226
==============================================================================
Omnibus:                        0.415   Durbin-Watson:                   0.779
Prob(Omnibus):                  0.813   Jarque-Bera (JB):                0.204
Skew:                           0.057   Prob(JB):                        0.903
Kurtosis:                       3.106   Cond. No.                     8.61e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.61e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""